# Feast Operator with RBAC Configuration
## Objective

This demo provides a reference implementation of a runbook on how to enable Role-Based Access Control (RBAC) for Feast using the Feast Operator with the Kubernetes authentication type. This serves as useful reference material for a cluster admin / MLOps engineer.

The demo steps include deploying the Feast Operator, creating Feast instances with server components (registry, offline store, online store), and Feast client testing locally and fom the Kubernetes. The goal is to ensure secure access control for Feast instances deployed by the Feast Operator.
 
Please read these reference documents for understanding the Feast RBAC framework.
- [RBAC Architecture](https://docs.feast.dev/v/master/getting-started/architecture/rbac) 
- [RBAC Permission](https://docs.feast.dev/v/master/getting-started/concepts/permission).
- [RBAC Authorization Manager](https://docs.feast.dev/v/master/getting-started/components/authz_manager)


## Deployment Architecture
In this notebook, we will deploy a distributed topology of Feast services, which includes:

* `Registry Server`: Handles metadata storage for feature definitions.
* `Online Store Server`: Uses the `Registry Server` to query metadata and is responsible for low-latency serving of features.
* `Offline Store Server`: Uses the `Registry Server` to query metadata and provides access to batch data for historical feature retrieval.

Additionally, we will cover:
* RBAC Configuration with Kubernetes Authentication for Feast resources.

## Prerequisites
* Kubernetes Cluster
* [kubectl](https://kubernetes.io/docs/tasks/tools/#kubectl) Kubernetes CLI tool.

## Install Prerequisites
The following commands install and configure all the prerequisites on a MacOS environment. You can find the
equivalent instructions on the offical documentation pages:
* Install the `kubectl` cli.
* Install Kubernetes and Container runtime (e.g. [Colima](https://github.com/abiosoft/colima)).
  * Alternatively, authenticate to an existing Kubernetes or OpenShift cluster.
  
```bash
brew install colima kubectl
colima start -r containerd -k -m 3 -d 100 -c 2 --cpu-type max -a x86_64
colima list
```

In [1]:
!kubectl create ns feast
!kubectl config set-context --current --namespace feast

namespace/feast created
Context "kind-kind" modified.


Validate the cluster setup:

In [2]:
!kubectl get ns feast

NAME    STATUS   AGE
feast   Active   6s


## Feast Admin Steps:
Feast Admins or MLOps Engineers may require Kubernetes Cluster Admin roles when working with OpenShift or Kubernetes clusters. Below is the list of steps Required to set up Feast RBAC with the Operator by an Admin or MLOps Engineer.

1. **Install the Feast Operator**
2. **Install the Feast services via FeatureStore CR**
3. **Configure the RBAC Permissions**
4. **Perform Feast Apply**
5. **Setting Service Account and Role Binding**

## Install the Feast Operator

In [3]:
## Use this install command from a stable branch  
!kubectl apply -f ../../infra/feast-operator/dist/install.yaml

## OR, for the latest code/builds, use one the following commands from the 'master' branch
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:develop FS_IMG=quay.io/feastdev-ci/feature-server:develop
# !make -C ../../infra/feast-operator install deploy IMG=quay.io/feastdev-ci/feast-operator:$(git rev-parse HEAD) FS_IMG=quay.io/feastdev-ci/feature-server:$(git rev-parse HEAD)

!kubectl wait --for=condition=available --timeout=5m deployment/feast-operator-controller-manager -n feast-operator-system

namespace/feast-operator-system created
customresourcedefinition.apiextensions.k8s.io/featurestores.feast.dev created
serviceaccount/feast-operator-controller-manager created
role.rbac.authorization.k8s.io/feast-operator-leader-election-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-editor-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-featurestore-viewer-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-manager-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-auth-role created
clusterrole.rbac.authorization.k8s.io/feast-operator-metrics-reader created
rolebinding.rbac.authorization.k8s.io/feast-operator-leader-election-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-manager-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/feast-operator-metrics-auth-rolebinding created
service/feast-operator-controller-manager-metrics-service created
deployment.ap

## Install the Feast services via FeatureStore CR
Next, we'll use the running Feast Operator to install the feast services with Server components online, offline, registry with kubernetes Authorization set. Apply the included [reference deployment](../../infra/feast-operator/config/samples/v1alpha1_featurestore_kubernetes_auth.yaml) to install and configure Feast with kubernetes Authorization .

In [4]:
!cat ../../infra/feast-operator/config/samples/v1alpha1_featurestore_kubernetes_auth.yaml
!kubectl apply -f ../../infra/feast-operator/config/samples/v1alpha1_featurestore_kubernetes_auth.yaml -n feast

apiVersion: feast.dev/v1alpha1
kind: FeatureStore
metadata:
  name: sample-kubernetes-auth
spec:
  feastProject: feast_rbac
  authz:
    kubernetes:
      roles:
        - feast-writer
        - feast-reader
  services:
    offlineStore:
      server: {}
    onlineStore:
      server: {}
    registry:
      local:
        server: {}
    ui: {}
featurestore.feast.dev/sample-kubernetes-auth created


## Validate the running FeatureStore deployment
Validate the deployment status.

In [6]:
!kubectl get all
!kubectl wait --for=condition=available --timeout=8m deployment/feast-sample-kubernetes-auth

NAME                                                READY   STATUS    RESTARTS   AGE
pod/feast-sample-kubernetes-auth-775774f6f6-t8q4x   4/4     Running   0          28s

NAME                                            TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/feast-sample-kubernetes-auth-offline    ClusterIP   10.96.98.188    <none>        80/TCP    28s
service/feast-sample-kubernetes-auth-online     ClusterIP   10.96.181.225   <none>        80/TCP    28s
service/feast-sample-kubernetes-auth-registry   ClusterIP   10.96.95.48     <none>        80/TCP    28s
service/feast-sample-kubernetes-auth-ui         ClusterIP   10.96.243.121   <none>        80/TCP    28s

NAME                                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/feast-sample-kubernetes-auth   1/1     1            1           28s

NAME                                                      DESIRED   CURRENT   READY   AGE
replicaset.apps/feast-sample-kubernetes-auth-7757

Validate that the FeatureStore CR is in a `Ready` state.

In [7]:
!kubectl get feast

NAME                     STATUS   AGE
sample-kubernetes-auth   Ready    37s


## Configure the RBAC Permissions
As we have created Kubernetes roles in FeatureStore CR to manage access control for Feast objects, the Python script `permissions_apply.py` will apply these roles to configure permissions. See the detailed code example below with comments.

In [8]:
#view the permissions  
!cat  permissions_apply.py

# Necessary modules for permissions and policies in Feast for RBAC
from feast.feast_object import ALL_RESOURCE_TYPES
from feast.permissions.action import READ, AuthzedAction, ALL_ACTIONS
from feast.permissions.permission import Permission
from feast.permissions.policy import RoleBasedPolicy

# Define K8s roles same as created with FeatureStore CR
admin_roles = ["feast-writer"]  # Full access (can create, update, delete ) Feast Resources
user_roles = ["feast-reader"]   # Read-only access on Feast Resources

# User permissions (feast_user_permission)
# - Grants read and describing Feast objects access
user_perm = Permission(
    name="feast_user_permission",
    types=ALL_RESOURCE_TYPES,
    policy=RoleBasedPolicy(roles=user_roles),
    actions=[AuthzedAction.DESCRIBE] + READ  # Read access (READ_ONLINE, READ_OFFLINE) + describe other Feast Resources.
)

# Admin permissions (feast_admin_permission)
# - Grants full control over all resources
admin_perm = Permission(
    name="feast_admin_

In [9]:
# Copy the Permissions to the pods under feature_repo directory
!kubectl cp permissions_apply.py $(kubectl get pods -l 'feast.dev/name=sample-kubernetes-auth' -ojsonpath="{.items[*].metadata.name}"):/feast-data/feast_rbac/feature_repo -c online

In [10]:
#view the feature_store.yaml configuration 
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- cat feature_store.yaml

project: feast_rbac
provider: local
offline_store:
    type: dask
online_store:
    path: /feast-data/online_store.db
    type: sqlite
registry:
    path: /feast-data/registry.db
    registry_type: file
auth:
    type: kubernetes
entity_key_serialization_version: 3


## Apply the Permissions and Feast Object to Registry

In [11]:
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast apply

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_len" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/feast-data/feast_rbac/feature_repo/example_repo.py:27: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity 'driver'.
  driver = Entity(name="driver", join_keys=["driver_id"])
Applying changes for project feast_rbac
/opt/app-root/lib64/python3.11/site-packages/feast/feature_store.py:581: RuntimeWarning: On demand feature view is an experimental feature. Thi

**List the applied permission details permissions on Feast Resources.**

In [12]:
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast permissions list-roles
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast permissions list
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast permissions describe feast_admin_permission
!kubectl exec deploy/feast-sample-kubernetes-auth -itc online -- feast permissions describe feast_user_permission

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_len" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
+--------------+
| ROLE NAME    |
+==============+
| feast-reader |
+--------------+
| feast-writer |
+--------------+
<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows a

## Setting Up Service Account and RoleBinding 
The steps below will:
- Create **three different ServiceAccounts** for Feast.
- Assign appropriate **RoleBindings** for access control.

## Test Cases
| User Type       | ServiceAccount               | RoleBinding Assigned | Expected Behavior in output                                |
|----------------|-----------------------------|----------------------|------------------------------------------------------------|
| **Read-Only**  | `feast-user-sa`              | `feast-reader`       | Can **read** from the feature store, but **cannot write**. |
| **Unauthorized** | `feast-unauthorized-user-sa` | _None_               | **Access should be denied** in `test.py`.                  |
| **Admin**      | `feast-admin-sa`             | `feast-writer`       | Can **read and write** feature store data.                 |

### Set Up a Read-Only Feast User 
(ServiceAccount: feast-user-sa, Role: feast-reader)

In [13]:
# Step 1: Create the ServiceAccount
!echo "Creating ServiceAccount: feast-user-sa"
!kubectl create serviceaccount feast-user-sa -n feast

# Step 2: Assign RoleBinding (Read-Only Access for Feast)
!echo "Assigning Read-Only RoleBinding: feast-user-rolebinding"
!kubectl create rolebinding feast-user-rolebinding --role=feast-reader --serviceaccount=feast:feast-user-sa -n feast

Creating ServiceAccount: feast-user-sa
serviceaccount/feast-user-sa created
Assigning Read-Only RoleBinding: feast-user-rolebinding
rolebinding.rbac.authorization.k8s.io/feast-user-rolebinding created


### Set Up an Unauthorized Feast User
(ServiceAccount: feast-unauthorized-user-sa, Role: None)

In [14]:
# Create the ServiceAccount (Without RoleBinding)
!echo "Creating Unauthorized ServiceAccount: feast-unauthorized-user-sa"
!kubectl create serviceaccount feast-unauthorized-user-sa -n feast


Creating Unauthorized ServiceAccount: feast-unauthorized-user-sa
serviceaccount/feast-unauthorized-user-sa created


### Set Up a Test Admin Feast User
(ServiceAccount: feast-admin-sa, Role: feast-writer)

In [15]:
# Create the ServiceAccount
!echo "Creating ServiceAccount: feast-admin-sa"
!kubectl create serviceaccount feast-admin-sa -n feast

# Assign RoleBinding (Admin Access for Feast)
!echo "Assigning Admin RoleBinding: feast-admin-rolebinding"
!kubectl create rolebinding feast-admin-rolebinding --role=feast-writer --serviceaccount=feast:feast-admin-sa -n feast


Creating ServiceAccount: feast-admin-sa
serviceaccount/feast-admin-sa created
Assigning Admin RoleBinding: feast-admin-rolebinding
rolebinding.rbac.authorization.k8s.io/feast-admin-rolebinding created


[Next: Client example from Pod](./2-client-rbac-test-pod.ipynb)